In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

In [2]:
with open('utilities/lookup_users.pkl', 'rb') as f:
    full_lookup = pickle.load(f)

df = pd.read_pickle('gitshared_csv/matteosalviniofficial/matteosalviniofficial_comments_20230316_0-1100_preprocessed.df')

Backing off send_request(...) for 71.7s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000267E6B6A4F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))
